# ChatBot

In [1]:
import pickle
import numpy as np

In [2]:
with open("train_qa.txt", "rb") as fp:   # Unpickling
    train_data =  pickle.load(fp)

In [3]:
with open("test_qa.txt", "rb") as fp:   # Unpickling
    test_data =  pickle.load(fp)

In [4]:
len(test_data)

1000

In [5]:
len(train_data)

10000

In [6]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [8]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [10]:
' '.join(train_data[0][2])

'n o'

In [11]:
all_data=test_data+train_data

In [12]:
len(all_data)

11000

In [13]:
vocab = set()
for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
vocab.add('no')
vocab.add('yes')

In [16]:
vocab_len=len(vocab)+1

In [17]:
vocab_len

38

In [21]:
all_story_len=[len(data[0]) for data in all_data]
max_story_len = max(all_story_len)
max_story_len

156

In [22]:
max_question_len = max([len(data[1]) for data in all_data])
max_question_len

6

## Vectorizing the Data

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [26]:
tokenizer = Tokenizer(filters=[])  #create an instance of our tokenizer
tokenizer.fit_on_texts(vocab)

In [27]:
tokenizer.word_index

{'picked': 1,
 'journeyed': 2,
 'got': 3,
 'office': 4,
 'up': 5,
 'milk': 6,
 'to': 7,
 'travelled': 8,
 'sandra': 9,
 'the': 10,
 'discarded': 11,
 'daniel': 12,
 'john': 13,
 'went': 14,
 'football': 15,
 'put': 16,
 'hallway': 17,
 'left': 18,
 'yes': 19,
 'took': 20,
 'down': 21,
 '?': 22,
 'no': 23,
 'garden': 24,
 'is': 25,
 'there': 26,
 'back': 27,
 'bedroom': 28,
 'kitchen': 29,
 'apple': 30,
 'moved': 31,
 'bathroom': 32,
 'mary': 33,
 'in': 34,
 '.': 35,
 'grabbed': 36,
 'dropped': 37}

In [28]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [29]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [31]:
len(train_story_seq)

10000

In [32]:
len(train_story_text)

10000

### Functionalize Vectorization

In [36]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_question_len=max_question_len):
    '''
    INPUT: 
    
    data: consisting of Stories,Queries,and Answers
    word_index: word index dictionary from tokenizer
    max_story_len: the length of the longest story (used for pad_sequences function)
    max_question_len: length of the longest question (used for pad_sequences function)


    OUTPUT:
    
    Vectorizes the stories,questions, and answers into padded sequences. We first loop for every story, query , and
    answer in the data. Then we convert the raw words to an word index value. Then we append each set to their appropriate
    output list. Then once we have converted the words to numbers, we pad the sequences so they are all of equal length.
    
    Returns this in the form of a tuple (X,Xq,Y) (padded based on max lengths)
    '''
    
    
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [37]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [38]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [39]:
inputs_test

array([[ 0,  0,  0, ..., 10, 28, 35],
       [ 0,  0,  0, ..., 10, 24, 35],
       [ 0,  0,  0, ..., 10, 24, 35],
       ...,
       [ 0,  0,  0, ..., 10, 30, 35],
       [ 0,  0,  0, ..., 10, 24, 35],
       [ 0,  0,  0, ..., 30, 26, 35]])

In [40]:
queries_test

array([[25, 13, 34, 10, 29, 22],
       [25, 13, 34, 10, 29, 22],
       [25, 13, 34, 10, 24, 22],
       ...,
       [25, 33, 34, 10, 28, 22],
       [25,  9, 34, 10, 24, 22],
       [25, 33, 34, 10, 24, 22]])

In [41]:
sum(answers_test)#there's index 37-497 no and index12-503 yes

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,   0.,   0.,
         0., 503.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.])